<a href="https://colab.research.google.com/github/menelaosas/DL_ASV_PROJECT/blob/main/deep_l.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Data preprocess to convert audio signsls into spectrograms using the librosa library


In [ ]:
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt

def create_spectrogram(audio_file, n_fft=2048, hop_length=512, resize_shape=(64,64)):
    # Load the audio file
    y, sr = librosa.load(audio_file)

    # Create a spectrogram
    D = librosa.stft(y, n_fft=n_fft, hop_length=hop_length)

    # Convert to magnitude spectrogram
    spect, phase = librosa.magphase(D)

    # Convert to dB scale
    spect_db = librosa.amplitude_to_db(spect, ref=np.max)

    # Resize the spectrogram
    spect_db_resized = resize(spect_db, resize_shape)

    return spect_db_resized

# Display the spectrogram
spectrogram = create_spectrogram('path_to_audio_file.wav')
plt.figure(figsize=(10, 4))
librosa.display.specshow(spectrogram, x_axis='time', y_axis='log')
plt.colorbar(format='%+2.0f dB')
plt.title('Spectrogram')
plt.show()


Generative Adversarial Networks (GANs)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Reshape, Conv2DTranspose, Conv2D, Flatten
from tensorflow.keras.optimizers import Adam

# Define the generator
def build_generator(z_dim):
    model = Sequential()
    model.add(Dense(256 * 16 * 16, input_dim=z_dim))
    model.add(Reshape((16, 16, 256)))
    model.add(Conv2DTranspose(128, kernel_size=3, strides=2, padding='same'))
    model.add(Conv2DTranspose(64, kernel_size=3, strides=2, padding='same'))
    model.add(Conv2DTranspose(1, kernel_size=3, strides=2, padding='same', activation='tanh'))
    return model

# Define the discriminator
def build_discriminator(img_shape):
    model = Sequential()
    model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=img_shape, padding='same'))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding='same'))
    model.add(Conv2D(128, kernel_size=3, strides=2, padding='same'))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

# Define the GAN
def build_gan(generator, discriminator):
    model = Sequential()
    model.add(generator)
    model.add(discriminator)
    return model

# Spectrogram shape depends on your data
spectrogram_shape = (64, 64, 1)

# Build and compile the discriminator
discriminator = build_discriminator(spectrogram_shape)
discriminator.compile(loss='binary_crossentropy', optimizer=Adam(), metrics=['accuracy'])

# Build the generator
generator = build_generator(100)

# Keep discriminator's parameters constant for generator training
discriminator.trainable = False

# Build and compile the GAN
gan = build_gan(generator, discriminator)
gan.compile(loss='binary_crossentropy', optimizer=Adam())

# Load your data and preprocess it
# ...

# Train your GAN
# ...


Efficacy of GAN-based Spoofing Attacks

In [ ]:
# Assume `gan_spoof_samples` is a NumPy array of your GAN-generated spoof samples
gan_spoof_samples = preprocess_data(gan_spoof_samples) # Preprocess these samples the same way you did with your training data

# Assume these samples are all spoofed
y_true = np.ones(len(gan_spoof_samples))

# Use the ASV system to predict whether each GAN-generated sample is genuine or spoofed
y_pred = np.argmax(model.predict(gan_spoof_samples), axis=1)

# Compute the error rate
error_rate = np.mean(y_true != y_pred)

print("ASV Error Rate on GAN-generated samples: ", error_rate)

CNN-based Anti-Spoofing Measures

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split

# Assume X is your preprocessed data and Y is your labels
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Convert the labels to one-hot encoding
Y_train = to_categorical(Y_train, num_classes=2)
Y_test = to_categorical(Y_test, num_classes=2)

# Define the CNN architecture
def build_model(input_shape):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', input_shape=input_shape))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Conv2D(64, (3, 3), activation='relu'))
    model.add(Flatten())
    model.add(Dense(64, activation='relu'))
    model.add(Dense(2, activation='softmax')) # output layer, 2 nodes for 2 classes
    return model

# Spectrogram shape depends on your data
spectrogram_shape = (64, 64, 1)

# Build the model
model = build_model(spectrogram_shape)

# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=0.001),
              metrics=['accuracy'])

# Train the model
history = model.fit(X_train, Y_train,
                    validation_data=(X_test, Y_test),
                    epochs=20, batch_size=32)


# Investigate the Performance of CNN-based ASV System


In [ ]:
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import roc_curve, auc
import matplotlib.pyplot as plt
import numpy as np

# Predict classes using the model
y_pred = model.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)

# Convert Y_test to 1D array
Y_test_classes = np.argmax(Y_test, axis=1)

# Calculate metrics
print(classification_report(Y_test_classes, y_pred_classes))

# Generate confusion matrix
print(confusion_matrix(Y_test_classes, y_pred_classes))

# ROC curve and AUC
fpr, tpr, _ = roc_curve(Y_test_classes, y_pred_classes)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.plot(fpr, tpr, color='darkorange', lw=2, label='ROC curve (area = %0.2f)' % roc_auc)
plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic')
plt.legend(loc="lower right")
plt.show()

# Plot training & validation accuracy values
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()

# Plot training & validation loss values
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')
plt.show()


Effectiveness of CNN-based Anti-Spoofing Measures

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score.

# Assume 'X_val' is your validation data and 'y_val' are the corresponding true labels
# Preprocess validation data
X_val_preprocessed = preprocess_data(X_val)

# Make predictions on the validation data
y_pred = np.argmax(model.predict(X_val_preprocessed), axis=1)

# Compute true labels from one-hot encoded to class labels
y_true = np.argmax(y_val, axis=1)

# Compute performance metrics
accuracy = accuracy_score(y_true, y_pred)
precision = precision_score(y_true, y_pred)
recall = recall_score(y_true, y_pred)
f1 = f1_score(y_true, y_pred)

print(f"Accuracy: {accuracy}")
print(f"Precision: {precision}")
print(f"Recall: {recall}")
print(f"F1-Score: {f1}")
print(f"AUC-ROC: {roc_auc}")


Comparative Analysis and Interpretation of Results of CNN and GANs

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

# Get the predictions for the test set from each model
cnn_predictions = cnn_model.predict(X_test)
gan_predictions = gan_model.predict(X_test)

# Calculate the accuracy of each model
cnn_accuracy = accuracy_score(y_test, cnn_predictions)
gan_accuracy = accuracy_score(y_test, gan_predictions)

print(f"Accuracy of CNN model: {cnn_accuracy}")
print(f"Accuracy of GAN model: {gan_accuracy}")

# Compare the models' accuracies in a bar chart
labels = ['CNN', 'GAN']
accuracies = [cnn_accuracy, gan_accuracy]

x = np.arange(len(labels))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots()
rects = ax.bar(x, accuracies, width)

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Accuracy')
ax.set_title('Comparative Analysis of CNN and GAN Accuracy')
ax.set_xticks(x)
ax.set_xticklabels(labels)

fig.tight_layout()

plt.show()
